# Veri Tipleri

Spark üzerinde halihazıda temel işlemleri bitirdik dataFrame'ler üzerindeki temel işlemlere gelmeden spark üzerindeki diğer veri tiplerine bir göz atalım.

###### boolean

In [1]:
WorldCupData=spark.read.format("csv").option("inferSchema","true").option("header","true").load("WorldCupMatches.csv")

In [2]:
WorldCupData.schema

StructType(List(StructField(Year,IntegerType,true),StructField(Datetime,StringType,true),StructField(Stage,StringType,true),StructField(Stadium,StringType,true),StructField(City,StringType,true),StructField(Home Team Name,StringType,true),StructField(Home Team Goals,IntegerType,true),StructField(Away Team Goals,IntegerType,true),StructField(Away Team Name,StringType,true),StructField(Win conditions,StringType,true),StructField(Attendance,IntegerType,true),StructField(Half-time Home Goals,IntegerType,true),StructField(Half-time Away Goals,IntegerType,true),StructField(Referee,StringType,true),StructField(Assistant 1,StringType,true),StructField(Assistant 2,StringType,true),StructField(RoundID,IntegerType,true),StructField(MatchID,IntegerType,true),StructField(Home Team Initials,StringType,true),StructField(Away Team Initials,StringType,true)))

In [4]:
from pyspark.sql.functions import col,struct,array,concat,lit
df=WorldCupData.selectExpr("Year as Yıl","`Home team name` as Evsahibi","`Home Team Goals` as EvsahibiGol"\
                           ,"`Away Team Goals` as DeplasmanGol","`Away Team Name` as Deplasman","City as sehir")
df.show()

+----+----------+-----------+------------+----------+-----------+
| Yıl|  Evsahibi|EvsahibiGol|DeplasmanGol| Deplasman|      sehir|
+----+----------+-----------+------------+----------+-----------+
|1930|    France|          4|           1|    Mexico|Montevideo |
|1930|       USA|          3|           0|   Belgium|Montevideo |
|1930|Yugoslavia|          2|           1|    Brazil|Montevideo |
|1930|   Romania|          3|           1|      Peru|Montevideo |
|1930| Argentina|          1|           0|    France|Montevideo |
|1930|     Chile|          3|           0|    Mexico|Montevideo |
|1930|Yugoslavia|          4|           0|   Bolivia|Montevideo |
|1930|       USA|          3|           0|  Paraguay|Montevideo |
|1930|   Uruguay|          1|           0|      Peru|Montevideo |
|1930|     Chile|          1|           0|    France|Montevideo |
|1930| Argentina|          6|           3|    Mexico|Montevideo |
|1930|    Brazil|          4|           0|   Bolivia|Montevideo |
|1930|  Pa

şimdi gördüğünüz üzere çok fazla parametre tutan dataframimizi basit bir transformation ile daha ufak üzerinde daha rahat antrenman yapabileceğimiz bir hale getirdirdik.

In [6]:
df.printSchema()
df.where("yıl==1938").show()#basit bir where kullanımı

root
 |-- Yıl: integer (nullable = true)
 |-- Evsahibi: string (nullable = true)
 |-- EvsahibiGol: integer (nullable = true)
 |-- DeplasmanGol: integer (nullable = true)
 |-- Deplasman: string (nullable = true)
 |-- sehir: string (nullable = true)

+----+--------------+-----------+------------+-----------------+-----------+
| Yıl|      Evsahibi|EvsahibiGol|DeplasmanGol|        Deplasman|      sehir|
+----+--------------+-----------+------------+-----------------+-----------+
|1938|   Switzerland|          1|           1|          Germany|     Paris |
|1938|       Hungary|          6|           0|Dutch East Indies|     Reims |
|1938|        France|          3|           1|          Belgium|  Colombes |
|1938|          Cuba|          3|           3|          Romania|  Toulouse |
|1938|         Italy|          2|           1|           Norway|Marseilles |
|1938|        Brazil|          6|           5|           Poland|Strasbourg |
|1938|Czechoslovakia|          3|           0|      Nether

Gelin birde sutün üzerinde nasıl kısıtlamalar yapacağımıza bakalım

Şimdi alttaki kod öbeğini anlatalım şehir filtresi için 'Sao' kelimesini içersinde 1 ve ya daha fazla barındıran satırları.
<p>Ev sahibi filtresi içinde aynı durum geçerli ve son filtre olan gol filtresi ise 1 deplasmanda atılan gollerin 1 ve ya birden büyük olduğu durumlardaki satırları getirecek. 
   <p> ve en son where komutu içerisinde hepsini rahatlıkla kullanabileceğimizi görüyorsunuz.

In [7]:
from pyspark.sql.functions import instr
sehirFilter=instr(df.sehir,"Sao")>=1
evSahibiFilter=instr(df.Evsahibi,"Swe")>=1
golFilter=col("DeplasmanGol")>1
df.where(sehirFilter & evSahibiFilter & golFilter).show()





+----+--------+-----------+------------+---------+----------+
| Yıl|Evsahibi|EvsahibiGol|DeplasmanGol|Deplasman|     sehir|
+----+--------+-----------+------------+---------+----------+
|1950|  Sweden|          3|           2|    Italy|Sao Paulo |
+----+--------+-----------+------------+---------+----------+



In [59]:
conditions=sehirFilter & evSahibiFilter & golFilter
df.withColumn("ConditionState",conditions).where("ConditionState==true").show()

+----+--------+-----------+------------+---------+----------+--------------+
| Yıl|Evsahibi|EvsahibiGol|DeplasmanGol|Deplasman|     sehir|ConditionState|
+----+--------+-----------+------------+---------+----------+--------------+
|1950|  Sweden|          3|           2|    Italy|Sao Paulo |          true|
+----+--------+-----------+------------+---------+----------+--------------+



Bir yukarıda gördüğünüz kod parçası ise yıkarıda oluşturduğumuz filtreleri sutün üzerinde tekrar uygulayarak her sutündaki ilgili şarta göre boolean değer olarak ek bir sutün haline getirebiliriz yani istediğimiz şartı sağlayan sutünları boolean bir değerle etiketleyebiliriz.
Son bir ekleme eğer verinizin içersinde null değerler varsa onu eqNullSafe("None") ile düzeltebilirsiniz

###### nümerik ve string data işlemleri

In [21]:
from pyspark.sql.functions import expr
toplamGol=col("EvsahibiGol")+col("DeplasmanGol")
df.select("*",toplamGol.alias("ToplamGol")).where("toplamGol>10").show()


+----+--------+-----------+------------+-----------+-----------+---------+
| Yıl|Evsahibi|EvsahibiGol|DeplasmanGol|  Deplasman|      sehir|ToplamGol|
+----+--------+-----------+------------+-----------+-----------+---------+
|1938|  Brazil|          6|           5|     Poland|Strasbourg |       11|
|1954| Hungary|          8|           3| Germany FR|     Basel |       11|
|1954| Austria|          7|           5|Switzerland|  Lausanne |       12|
|1982| Hungary|         10|           1|El Salvador|     Elche |       11|
+----+--------+-----------+------------+-----------+-----------+---------+



Gördüğünüz gibi spark üzerindeki iki sutün eğer tipleri uygunsa üzerinde rahatlıkla 4 işlem yapmaya müsait hale geliyor,yukarıda iki col toplamıda bir col olacağıundan aşagıdaki select attığımız zaman ilgili data frame toplam gol sutünuna sahip olacaktır.

###### string

şimdi aşağıda hızlaca sütunlardaki stringler üzerindeki string manipülasyonlarını hızlıca örneklerle gösterip geçicem.

In [22]:
from pyspark.sql.functions import  lit,ltrim,rtrim,rpad,lpad,trim,upper
df.select(upper(df.Deplasman)).show(3)#ltrim,trim,rpad,lpad sağ ve ya soldan boşluk ekleme ve kesmeler için kullanılır.

+----------------+
|upper(Deplasman)|
+----------------+
|          MEXICO|
|         BELGIUM|
|          BRAZIL|
+----------------+
only showing top 3 rows



En son terki diyar etmeden önce string işlemleri ile ilgili ufak bir regular expression örneği verip kendimize sövdürmeyelim.

In [23]:
from pyspark.sql.functions import  regexp_replace
reg_string="France"
df.select(regexp_replace(col("Evsahibi"),reg_string,"Antep").alias("x"),col("Evsahibi")).show(2)

+-----+--------+
|    x|Evsahibi|
+-----+--------+
|Antep|  France|
|  USA|     USA|
+-----+--------+
only showing top 2 rows



### ilgili daha fazla fonksiyonlar için https://spark.apache.org/docs/1.6.2/api/java/org/apache/spark/sql/functions.html göz atabilirsiniz

###### Complex veri tipleri ile çalışma

spark üzerinde kullanabileceğimiz 3 farklı  komplex veri tipi vardır.
bunlar:structs,arrays,maps

Struct'lar DataFrame içinde dataFrame olarak düşünülebilinir.

In [63]:
structEx=df.selectExpr("struct(EvsahibiGol,DeplasmanGol) as skor","*")
#gördüğünüz üzere bir sql sorgusu ile bir struct oluştrduk.



Array'ler ise bazı fonksiyonlar ile sütunlardan kendileri olulurlar mesela gelin şehir sütununu birer "split" fonksiyonuyla birer arraye dönüştürelim.,

In [24]:
from pyspark.sql.functions import split,size
df.select(split(col("sehir"),"")).show(1)
df.select(size(split(col("sehir"),""))).show(1)
#diğer array fonksiyonlarıda aynı şekilde size gibi kullanarak deneyebilirsiniz.

+--------------------+
|      split(sehir, )|
+--------------------+
|[M, o, n, t, e, v...|
+--------------------+
only showing top 1 row

+--------------------+
|size(split(sehir, ))|
+--------------------+
|                  12|
+--------------------+
only showing top 1 row



Şimdi gelelim belkide spark üzerinde yapacağımız en kritik şeye kullanıcı tanımlı fonksiyonlar.

#### UDF(user definied function)

şimdi gelelim sparkın bu alandaki faydalarına eğer fonksiyonları pyhton ile yazarsanız spark bir
tane pyhton processesi oluşturur işci node üzerinde ve tüm verileri pyhton'ın anlayacağı şekilde serialize eder
sonra pyhton kodununda satır satır çalıştırır sonra sonucu JVM'e dönderir 
ve spark pyhton processeine giren veriyi kontrol edemeyeceği için işci node'lar üzerinde hataya sürükleyebilir bu yüzden user defined fonksiyonlarımızı daha çok scala ve java dilinde kullanırsak verimliliği arttırabiliriz.<p>Aşağıdaki kod öbeğindede görebileceğiniz gibi bir pyhton fonksiyonundan hiçbir farkı yok biz fonksiyona bir sütun gönderiyoruz o ise tüm satırlardaki değerleri 3 ile üssü alınmış bir sutün dönderiyor. 

In [31]:
udfDf=spark.range(5).toDF("data")
def power3(value):
    return value**3

df.select(power3(col("DeplasmanGol")+col("Evsahibigol"))).show()
#Yukarıda ki fonksiyon da gördüğünüz gibi bir sütunu tamamıyla alıp değeri dönderebiliyor.

+--------------------------------------+
|POWER((DeplasmanGol + Evsahibigol), 3)|
+--------------------------------------+
|                                 125.0|
|                                  27.0|
|                                  27.0|
|                                  64.0|
|                                   1.0|
|                                  27.0|
|                                  64.0|
|                                  27.0|
|                                   1.0|
|                                   1.0|
|                                 729.0|
|                                  64.0|
|                                   1.0|
|                                  64.0|
|                                  64.0|
|                                 343.0|
|                                 343.0|
|                                 216.0|
|                                 125.0|
|                                 216.0|
+--------------------------------------+
only showing top

# AGGREGATİONS(Toplama)

Aggregation aslında gruplama,toplama verileri combine etme şeklinde düşünülenlebilinir.Mesela veriyi groupBy ile gruplama bir aggregationdur.spark üzerinde bize bu yetkinliği veren window,grouping set,rollup,cuybe gibi methodlar vardır şimdi birer birer bunların neler olduğuna bakalım.

In [32]:
df.count()#en basit aggregation foksiyonudur.

4572

tüm aggregation fonksiyonları için org.apache.spark.sql.functions package 'e gidebilirsiniz.

In [33]:
#COUNT
#When count is performing as count(*) it count nul values 
#also but if you count spesific colum it will not count the null values.
from pyspark.sql.functions import count,countDistinct,approxCountDistinct,first\
,last,min,max,sum,sumDistinct,avg,var_pop,stddev,kurtosis,skewness,corr,covar_pop,covar_samp

In [34]:
df.select(count("sehir")).show()

+------------+
|count(sehir)|
+------------+
|         852|
+------------+



In [35]:
#bu ise bize dünya kupalarının şu ana kadar kaç farklı şehirde oynandığı bilgisini veriyor.
df.select(countDistinct("sehir")).show()


+---------------------+
|count(DISTINCT sehir)|
+---------------------+
|                  151|
+---------------------+



In [36]:
#çok büyük verilerle çalışacağımız zaman bu distinct count işlemleri çok maliyetli olacağından
#işlemi bir yakınsama değerinde alıyoruz,
#şöyle düşünebiliriz ilk yüz satırda kaç farklı değer var ise genele uyarla.
df.select(approxCountDistinct("sehir")).show()

+----------------------------+
|approx_count_distinct(sehir)|
+----------------------------+
|                         155|
+----------------------------+



In [37]:
#firs and last,you can get first and last value from a Df
df.select(first("sehir")).show()
df.select(last("sehir")).show()

+-------------------+
|first(sehir, false)|
+-------------------+
|        Montevideo |
+-------------------+

+------------------+
|last(sehir, false)|
+------------------+
|              null|
+------------------+



In [38]:
#min and max :return min and max value from column.
df.select(max("EvSahibiGol"),min("DeplasmanGol")).show()


+----------------+-----------------+
|max(EvSahibiGol)|min(DeplasmanGol)|
+----------------+-----------------+
|              10|                0|
+----------------+-----------------+



In [39]:
#sum,sumDistinct
#ev sahipleri 1543 gol atarken deplasman takımları ise 871 kadar gol atabilmiş.
df.select(sum("EvSahibiGol"),sum("DeplasmanGol")).show()

+----------------+-----------------+
|sum(EvSahibiGol)|sum(DeplasmanGol)|
+----------------+-----------------+
|            1543|              871|
+----------------+-----------------+



In [40]:
#avg,variance,standart deviation
#atılan ortalama goller.
df.select(avg("EvsahibiGol"),avg("DeplasmanGol")).show()
df.select(var_pop("EvsahibiGol"),stddev("EvsahibiGol")).show()


+------------------+-----------------+
|  avg(EvsahibiGol)|avg(DeplasmanGol)|
+------------------+-----------------+
|1.8110328638497653|1.022300469483568|
+------------------+-----------------+

+--------------------+------------------------+
|var_pop(EvsahibiGol)|stddev_samp(EvsahibiGol)|
+--------------------+------------------------+
|   2.589878275915273|      1.6102551385229658|
+--------------------+------------------------+



In [41]:
#kısaca şimdilik ortalama değere göre  uzaklaşma türüne göre çarpıklık ve basıklık olarak bilin.

df.select(skewness("EvsahibiGol"),kurtosis("EvsahibiGol")).show()

+---------------------+---------------------+
|skewness(EvsahibiGol)|kurtosis(EvsahibiGol)|
+---------------------+---------------------+
|   1.3493586476038122|     2.61650781761948|
+---------------------+---------------------+



In [42]:
#Covariance,Correlation:
#bunlar ise verileri +1 ile -1 arasında ölçeklendirmek için kullandığımız methodlardır.
df.select(corr("EvsahibiGol","DeplasmanGol"),covar_samp("EvsahibiGol","DeplasmanGol"),covar_pop("EvsahibiGol","DeplasmanGol")).show()

+-------------------------------+-------------------------------------+------------------------------------+
|corr(EvsahibiGol, DeplasmanGol)|covar_samp(EvsahibiGol, DeplasmanGol)|covar_pop(EvsahibiGol, DeplasmanGol)|
+-------------------------------+-------------------------------------+------------------------------------+
|           0.012473988573227762|                 0.021845329714282765|                 0.02181968965593267|
+-------------------------------+-------------------------------------+------------------------------------+



In [43]:
#AGREGATING TO COMPLEX TYPES


In [46]:
#gropings
df.groupBy("Sehir","yıl").count().show()

+------------------+----+-----+
|             Sehir| yıl|count|
+------------------+----+-----+
|            Paris |1938|    3|
|           Toluca |1970|    4|
|            Gijon |1982|    3|
|        Nuremberg |2006|    5|
|          Hamburg |2006|    5|
|        Polokwane |2010|    4|
|        Stuttgart |1974|    4|
|             Oita |2002|    3|
|    Gelsenkirchen |1974|    5|
|       Norrk�Ping |1958|    3|
|Santiago De Chile |1962|   10|
|          Hanover |2006|    5|
|         Valencia |1982|    3|
|          Leipzig |2006|    5|
|       Sunderland |1966|    4|
|            Arica |1962|    7|
|        Monterrey |1986|    8|
|    San Francisco |1994|    6|
|           Lugano |1954|    1|
|             Rome |1990|    6|
+------------------+----+-----+
only showing top 20 rows



In [44]:
from pyspark.sql.functions import collect_set,collect_list,expr,count,column



In [45]:
df.agg(collect_set("Sehir"),collect_list("Sehir")).show()

+--------------------+--------------------+
|  collect_set(Sehir)| collect_list(Sehir)|
+--------------------+--------------------+
|[Mendoza , Saint-...|[Montevideo , Mon...|
+--------------------+--------------------+



In [60]:
#grouping with expression
df.groupBy("Sehir").agg(count("Yıl").alias("quan"),expr("count(Yıl)")).show()

+------------------+----+----------+
|             Sehir|quan|count(Yıl)|
+------------------+----+----------+
|            Daegu |   4|         4|
|            Paris |   9|         9|
|            Natal |   4|         4|
|    San Francisco |   6|         6|
|Santiago De Chile |  10|        10|
|       Eskilstuna |   1|         1|
|        La Coru�A |   3|         3|
|           Bilbao |   3|         3|
|           Geneva |   4|         4|
|         Le Havre |   1|         1|
|           Verona |   4|         4|
|             Kobe |   3|         3|
|            Solna |   8|         8|
|        Liverpool |   5|         5|
|          Gwangju |   3|         3|
|           Cuiaba |   4|         4|
|          Niigata |   3|         3|
|      Guadalajara |  17|        17|
|           Boston |   6|         6|
|           Madrid |   7|         7|
+------------------+----+----------+
only showing top 20 rows



In [61]:
#gropings with map
df.groupBy("Yıl").agg(expr("avg(EvsahibiGol)"),expr("stddev_pop(DeplasmanGol)")).show()

+----+------------------+------------------------+
| Yıl|  avg(EvsahibiGol)|stddev_pop(DeplasmanGol)|
+----+------------------+------------------------+
|1990|1.2884615384615385|      1.0348941574672084|
|1974|1.3421052631578947|      1.3795097286401221|
|2006|           1.34375|      1.0372693644251716|
|1978|2.0789473684210527|      0.6299320633466223|
|null|              null|                    null|
|1934| 2.823529411764706|      0.6655122646461624|
|1994|1.5961538461538463|      1.0124958908529371|
|2014|            1.2375|      1.5647983096872262|
|1930|3.2777777777777777|      0.8258927081843614|
|1938| 3.388888888888889|      1.1928283640879938|
|1950|3.1363636363636362|      0.8143851303258598|
|1966|           2.15625|      0.6959705453537528|
|1982|1.8653846153846154|      0.7945357023120615|
|1998|           1.53125|      1.0438388809461927|
|1970|              2.25|      0.8379429798619952|
|1958|2.5142857142857142|      1.0245954901166348|
|1954|4.1923076923076925|      

In [62]:
#window
from pyspark.sql.window import Window
from pyspark.sql.functions import col,to_date,desc,dense_rank,rank

In [63]:
dfWithDate=WorldCupData.withColumn("date",to_date(col("Datetime"),"MM/d/yyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")
#data kullanımınıda burada göreb,l,rs,n,z.

In [52]:
#Grouping Sets
dfNoNull=dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")
dfNoNull.show(1)

+----+--------------------+-------+-------+-----------+--------------+---------------+---------------+--------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+-------------------+-------+-------+------------------+------------------+----+
|Year|            Datetime|  Stage|Stadium|       City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|        Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|date|
+----+--------------------+-------+-------+-----------+--------------+---------------+---------------+--------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+-------------------+-------+-------+------------------+------------------+----+
|1930|13 Jul 1930 - 15:00 |Group 1|Pocitos|Montevideo |        France|         